### Setup


In [2]:
# !pip install torch transformers

In [3]:
# Define dataset

import torch

class ShakespeareDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        tokenizer,
        seq_len=256,
        max_samples=None,
        file_path="data/shakespeare.txt",
    ):
        self.tokenizer = tokenizer
        self.seq_len = seq_len

        # Read Shakespeare text
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()


        # Flatten to single sequence, tokeniz and reshape to (num_batches, seq_len) 
        tokens = self.tokenizer.encode(text)
        n_batches = len(tokens) // seq_len
        self.sequences = torch.tensor(tokens[:n_batches * seq_len], dtype=torch.long).reshape(n_batches, seq_len)
        if max_samples is not None:
            self.sequences = self.sequences[:max_samples]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        return {"input_ids": seq, "labels": seq}



### Train GPT2

In [12]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig


# Hyperparameters
batch_size = 32
seq_len = 32
learning_rate = 1e-4
num_epochs = 1
num_samples = 10

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model_llm = AutoModelForCausalLM.from_config(AutoConfig.from_pretrained("gpt2"))
dataset = ShakespeareDataset(tokenizer, seq_len=256, max_samples=num_samples)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.AdamW(model_llm.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}", total=len(train_dataloader))
    for batch in pbar:
        output = model_llm(**batch)
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        pbar.set_postfix(loss=loss.item())

    # Validation
    model_llm.eval()
    eval_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            output = model_llm(**batch)
            eval_loss += output.loss.item()
    eval_loss /= (len(val_dataloader) * batch_size)

    # Log
    sample_input_ids = torch.tensor(tokenizer.encode("Hello, how are you?"))
    sample_text = tokenizer.decode(model_llm.generate(sample_input_ids.reshape(1, -1), max_length=16)[0])
    print(f"[Epoch {epoch+1}] Val loss: {eval_loss:.2f} | Sample: {repr(sample_text)}")

        


Token indices sequence length is longer than the specified maximum sequence length for this model (338024 > 1024). Running this sequence through the model will result in indexing errors
Epoch 1: 0it [00:00, ?it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Epoch 1] Val loss: 0.35 | Sample: 'Hello, how are you?claveclaveclaveclaveclave caps caps caps caps caps'


In [14]:
sample_input_ids = torch.tensor(tokenizer.encode("Hello, how are you?"))
sample_text = tokenizer.decode(model_llm.generate(sample_input_ids.reshape(1, -1), max_length=16)[0])

# Save checkpoint
torch.save(
    {
        "epoch": epoch,
        "model_state_dict": model_llm.state_dict(),
        "val_loss": eval_loss,
    },
    f"checkpoints/gpt2-latest-dev.pth",
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Train PC

In [4]:
from transformers import AutoTokenizer

# 1. Shakespeare dataset
file_path = "../data/shakespeare/main.txt"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
seq_len = 8
batch_size = 512
max_samples = 2

dataset = ShakespeareDataset(tokenizer, seq_len=seq_len, max_samples=max_samples)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# visualize
for i in range(min(5, max_samples)):
    print(f"{repr(tokenizer.decode(dataset[i]['input_ids']))} | ids: {dataset[i]['input_ids']}")

Token indices sequence length is longer than the specified maximum sequence length for this model (338024 > 1024). Running this sequence through the model will result in indexing errors


'First Citizen:\nBefore we proceed any' | ids: tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597])
' further, hear me speak.\n\n' | ids: tensor([2252,   11, 3285,  502, 2740,   13,  198,  198])


In [8]:
from tqdm import tqdm
from spflow.meta import Scope
from spflow.modules.rat import RatSPN
from spflow.modules.leaf import  Binomial, Categorical
from spflow import log_likelihood

depth = 3
n_region_nodes = 5
num_leaves = 5
num_repetitions = 2
n_root_nodes = 1
num_feature = seq_len
# n = torch.tensor(16) # total count for binomial distribution
n = torch.tensor(len(tokenizer))

scope = Scope(list(range(0, num_feature)))

# rat_leaf_layer = Binomial(scope=scope, n=n, out_channels=num_leaves, num_repetitions=num_repetitions)
rat_leaf_layer = Categorical(scope=scope, K=n, out_channels=num_leaves, num_repetitions=num_repetitions)
rat = RatSPN(
    leaf_modules=[rat_leaf_layer],
    n_root_nodes=n_root_nodes,
    n_region_nodes=n_region_nodes,
    num_repetitions=num_repetitions,
    depth=depth,
    outer_product=True,
    split_halves=True,
)

In [9]:
optimizer = torch.optim.Adam(rat.parameters(), lr=1e-2)
n_epochs = 1_000
log_every = 500
for epoch in range(n_epochs):
    for step, batch in tqdm(enumerate(dataloader), leave=False, total=len(dataloader)):
        optimizer.zero_grad()
        data = batch['input_ids']
        ll = log_likelihood(rat, data)      # (B,)
        loss = -ll.mean()                         # NLL
        loss.backward()
        optimizer.step()
    if epoch % log_every == 0:
        print(f"[Epoch {epoch}] Loss {loss.item():.2f}")

  0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Epoch 0] Loss 86.13


[Epoch 500] Loss 7.97


In [11]:
from spflow import sample_with_evidence

batch = next(iter(dataloader))
X_tensor = batch['input_ids'].to(torch.float32)

# Sample with evidence
evidence = X_tensor[0]
evidence[:3] = torch.nan
evidence = evidence.unsqueeze(0)
samples = sample_with_evidence(rat, evidence, is_mpe=True)

print(repr(tokenizer.decode(samples[0].to(torch.long))))


'First Citizen: me speak.\n\n'


### Max decode

In [ ]:
import torch
from transformers import LogitsProcessorList

from transformers import LogitsProcessor

class HeuristicProcessor(LogitsProcessor):
    def __init__(self, model_pc):
        super().__init__()
        self.model_pc = model_pc

    def __call__(self, input_ids, scores):
        # does nothing
        return scores


model_llm_state_dict = torch.load("checkpoints/gpt2-latest-dev.pth")["model_state_dict"]
model_llm.load_state_dict(model_llm_state_dict)

model_spn_state_dict = torch.load("checkpoints/rat-latest-dev.pth")["model_state_dict"]
model_spn.load_state_dict(model_spn_state_dict)

model_llm.eval()
model_spn.eval()

sample_input_ids = torch.tensor(tokenizer.encode("Hello, how are you?"))


# Generate text (greedy)
greedy_dict = model_llm.generate(sample_input_ids.reshape(1, -1), max_new_tokens=32, do_sample=False, output_scores=True, return_dict_in_generate=True)
y_hat_greedy = torch.cat(
    (sample_input_ids[1:].reshape(1, -1), greedy_dict['sequences'][:, :-1]),
    dim=-1
)
nll_greedy = torch.cat(greedy_dict['scores'], dim=0).gather(dim=1, index=y_hat_greedy).sum()


# Generate text (heuristic)
processors = LogitsProcessorList([
    HeuristicProcessor(model_spn)
])
heur_dict = model_llm.generate(
    sample_input_ids.reshape(1, -1), 
    max_new_tokens=32, 
    do_sample=False, 
    output_scores=True, 
    return_dict_in_generate=True,
    logits_processor=processors,
)
y_hat_heur = torch.cat(
    (sample_input_ids[1:].reshape(1, -1), greedy_dict['sequences'][:, :-1]),
    dim=-1
)
nll_heur = torch.cat(heur_dict['scores'], dim=0).gather(dim=1, index=y_hat_heur).sum()


print(f"NLL greedy: {nll_greedy.item():.2f}")
print(f"NLL heuristic: {nll_heur.item():.2f}")
